In [1]:
import pickle
import json
import sys
import os
import string
import requests
import ast  # for string to list: ast.literal_eval()
from bs4 import BeautifulSoup
import bs4
import time
from datetime import datetime
from datetime import date
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
import warnings

In [2]:
data_dir = "/Users/vigadam/Dropbox/My Mac (MacBook-Air.local)/Documents/github/media_network/media_data/clean_text/2020/"
files = os.listdir(data_dir)
files.sort()
files.remove("covid_monthly_444origo.csv")
data_files = [data_dir + file for file in files]

In [ ]:
df = pd.read_csv("/Users/vigadam/Dropbox/My Mac (MacBook-Air.local)/Documents/github/media_network/media_data/clean_text/2020/all_site_2020.csv")

In [28]:
df["month"]=df["date"].str.split("-").apply(lambda x: x[1])
df["day"]=df["date"].str.split("-").apply(lambda x: x[2])

In [29]:
for i in tqdm(range(df.shape[0])):
    if len(df.loc[i,"month"]) == 1:
        df.loc[i,"month"] = "0" + df.loc[i,"month"]
    if len(df.loc[i,"day"]) == 1:
        df.loc[i,"day"] = "0" + df.loc[i,"day"]

100%|██████████| 224293/224293 [21:37<00:00, 172.81it/s]


In [30]:
df["year"]=df["year"].apply(str)

In [31]:
df["date"] = df.agg('{0[year]}-{0[month]}-{0[day]}'.format, axis=1)

In [32]:
df["date"].apply(len).value_counts()

10    224293
Name: date, dtype: int64

In [33]:
df.to_csv("/Users/vigadam/Dropbox/My Mac (MacBook-Air.local)/Documents/github/media_network/media_data/clean_text/2020/all_site_2020.csv")

In [44]:
for file in data_files:
    df = pd.read_csv(file,index_col=0)
    print(df.columns)

Index(['link', 'rovat', 'date', 'year', 'month', 'page', 'Title', 'Text',
       'Author', 'Tags'],
      dtype='object')
Index(['link', 'year', 'date', 'month', 'page', 'Title', 'Text', 'links',
       'Author', 'Category', 'Tags'],
      dtype='object')
Index(['link', 'rovat', 'Title', 'Text', 'Author', 'date', 'Tags'], dtype='object')
Index(['datum', '_mod_ts', '__properties', 'rovat', 'rovat_url', 'rovat_slug',
       'kiadvany', 'kiadvany_url', 'tags', 'actual_fb_like', 'ts', 'kep',
       'cim', 'ajanlo', 'alcim', 'url', 'korhatar_stamp', 'alnevek', 'lead',
       'month', 'page', 'Text', 'Author', 'Tags'],
      dtype='object')
Index(['link', 'rovat', 'felosztas', 'Title', 'links', 'Author', 'Text',
       'year', 'month', 'day', 'Tags', 'Source'],
      dtype='object')
Index(['link', 'date', 'month', 'rovat', 'page', 'Title', 'Text', 'links',
       'Author', 'Tags'],
      dtype='object')


In [64]:
_24hu = pd.read_csv(data_files[0],index_col=0)[["link","rovat","date","page","Title","Text","Author","Tags"]].rename(columns={"rovat":"category"})
_24hu.columns = [item.lower() for item in _24hu.columns]

In [164]:
_444 = pd.read_csv(data_files[1],index_col=0)[["link","Category","date","page","Title","Text","Author","Tags"]]
_444.columns = [item.lower() for item in _444.columns]

In [77]:
_888 = pd.read_csv(data_files[2],index_col=0)[["link","rovat","date","Title","Text","Author","Tags"]].rename(columns={"rovat":"category"})


_888["page"] = "888"
_888.columns = [item.lower() for item in _888.columns]

In [152]:
index = pd.read_csv(data_files[3],index_col=0).drop(columns=["tags"]).rename(columns={"rovat":"category","url":"link","_mod_ts":"date","cim":"title"}).reset_index(drop=True)
index.columns = [item.lower() for item in index.columns]

index = index[["link","date","category","title","author","text","tags","page"]]

index["date"] = index["date"].str.split(" ").apply(lambda x: x[0])

In [192]:
mno = pd.read_csv(data_files[4],index_col = 0)
mno.columns = [item.lower() for item in mno.columns]
mno = mno.rename(columns={"rovat":"category"}).dropna(subset=["text","year"]).reset_index(drop=True)

mno["year"] = mno["year"].apply(int).apply(str)
mno["month"] = mno["month"].apply(int).apply(str)
mno["day"] = mno["day"].apply(int).apply(str)

for i in tqdm(range(mno.shape[0])):
    mno.loc[i,"date"] = "-".join([mno.loc[i,"year"],mno.loc[i,"month"],mno.loc[i,"day"]]) 

100%|██████████| 45157/45157 [03:53<00:00, 193.55it/s]


In [193]:
mno = mno[["link","date","category","title","author","text","tags"]]
mno["page"] = "mno"

In [194]:
origo = pd.read_csv(data_files[5],index_col = 0).rename(columns={"rovat":"category"})
origo.columns = [item.lower() for item in origo.columns]
origo = origo[["link","date","category","title","author","text","tags","page"]]
origo["date"] = origo["date"].apply(lambda x: str(x)[0:4] + "-" + str(x)[4:6] +"-"+ str(x)[6:])

In [195]:
all_sites = pd.concat([_24hu,index,origo,_444,_888,mno])
all_sites["page"]=all_sites["page"].apply(str)

In [196]:
all_sites.isnull().sum()

link            0
category        0
date            0
page            0
title       15339
text        26094
author      33743
tags        29091
dtype: int64

In [199]:
all_sites["year"] = all_sites["date"].str.split("-").apply(lambda x: x[0])

In [205]:
all_sites = all_sites.loc[all_sites["year"] == "2020"].reset_index(drop=True)

In [209]:
all_sites.to_csv(data_dir + "all_site_2020.csv")